In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import betabinom
import math
from sklearn.datasets import make_s_curve
import random

In [ ]:
def price_sigmoid(x, max_x = 4000, a = 0.65, b = 10):
    s = 1/(1 + np.exp(b * (x / max_x - a)))
    return (s - min(s))*(1 - 0.9)/(max(s) - min(s)) + 0.9 # normalize function to 0.9-1

In [ ]:
def price_thresholds(x, max_discount = 0.1, last_price = 1000):
    if 0 <= x <= last_price * 0.1:
        price = 1
    elif last_price * 0.1 < x <= last_price * 0.4:
        price = 1 - max_discount * 0.1
    elif last_price * 0.4 < x <= last_price * 0.8:
        price = 1 - max_discount * 0.3
    elif last_price * 0.8 < x < last_price:
        price = 1 - max_discount * 0.7
    else:
        price = 1 - max_discount
    return price

In [ ]:
def simul(periods, agent, client_demand, suppliers_demands, 
              inventory_goal, storage_cost, penalty):
        
    sup = len(suppliers_demands)
    profit = {}; profit[0] = 0
    orig_profit = {}; orig_profit[0] = 0
    
    inventory = {}; inventory[0] = {'state': inventory_goal, 'cost': 0.8}
        
    results = {'0': {"profit" : 0, 
                     "inventory_state" : inventory[0]['state'], 
                     "inventory_cost" : inventory[0]['cost'],                   
                     "demand_distrib" : np.zeros(sup),
                     "ordered" : np.zeros(sup) , 
                     "mean_prices" : np.zeros(sup)} }
    
    # sigmoid price function:
    price_chart = price_sigmoid(np.linspace(0, inventory_goal, inventory_goal+1), max_x = client_demand)
    # threshold price function:
    #price_chart = [price_thresholds(i, max_discount = 0.1, last_price = client_demand) \
    #               for i in np.linspace(0, inventory_goal, inventory_goal+1)]
    
    for t in range(1, periods):

        client_demand_t = -1
        while (client_demand_t < 0):
            client_demand_t = np.random.normal(client_demand, client_demand/4)
        sup_production = abs(1 - betabinom.rvs(1, 0.1, 1, size = sup)) * 1000000
        if(inventory[t-1]['state'] <= 0.5 * inventory_goal):
            diff = inventory_goal - inventory[t-1]['state']
            demand_distrib = np.array([np.ceil(diff * suppliers_demands[s] / sum(suppliers_demands)) \
                                       for s in range(sup)])
            ordered = np.array([int(min(sup_production[s], demand_distrib[s])) for s in range(sup)])
            mean_prices = np.array([price_chart[i] * 0.8 for i in ordered])
        else:
            demand_distrib = np.zeros(sup)
            ordered = np.zeros(sup)
            mean_prices = np.zeros(sup)
            
        in_stock = inventory[t-1]['state'] + sum(ordered)

        if in_stock == 0:
            avrg_cost = 0
        else:
            avrg_cost = (inventory[t-1]['state'] * inventory[t-1]['cost'] + \
                         sum(ordered * mean_prices)) / in_stock        
        
        profit[t] = min(in_stock, client_demand_t) * (1 - avrg_cost) \
                        - inventory[t-1]['state'] * storage_cost \
                        - max(client_demand_t - in_stock, 0) * penalty
        
        
        inventory[t] = {'state': max(in_stock - client_demand_t, 0), 'cost': avrg_cost}
        orig_profit[t] = profit[t].copy()
        
        if agent == "agent":
            profit[t] = max(profit[t], 0)
        
        results[str(t)] = {"agent": agent,
                           "inventory_goal": inventory_goal,
                           "suppliers_demands": suppliers_demands,
                           "no_sup": sup,
                           "client_demand": client_demand_t,
                           "demand_distrib" : demand_distrib, 
                           "ordered" : ordered,
                           "mean_prices" : mean_prices,
                           "profit" : profit[t],
                           "inventory_state" : inventory[t]['state'], 
                           "inventory_cost" : inventory[t]['cost'],
                           "orig_profit" : orig_profit[t]}
                    
    results["final"] = sum(profit.values())/periods
    return(results)